<a href="https://colab.research.google.com/github/Kunslog/ML_summer_Sep_25/blob/main/template_conformal_inference2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# #2 Conformal inference

**Oxford University Economics Summer School 2025, Foundations of Machine Learning**

*Johanna Barop, 2025*

This is the soultion template for the second problem set for the *Foundations of Machine Learning* course at the **Oxford University Economics Summer School 2025**.


In this problem, you are asked to implement and evaluate some **conformal inference** procedures in Python. This problem builds on the first coding exercise, using Scikit-learn.

You may want to consult [Angelopoulos, A. N. and Bates, S. (2021)](https://arxiv.org/pdf/2107.07511), "A gentle introduction to conformal prediction and distribution-free uncertainty quantification".

# Conformal inference
Conformal inference is a method of uncertainty prediction. Normally we predict outcomes. Now, we will predict a _set_ that is guaranteed to contain the correct outcome in at least $(1-\alpha)$% of cases.







## How does conformal inference work intuitively?

The **goal** of conformal inference is uncertainty quantification through generating _prediction sets_.

![prediction-set-example.jpg](https://drive.google.com/uc?export=view&id=178JhIp_t4JkeToHP4x-q7frFGd4re_DN)

Figure taken from: Angelopoulos, A. N. and Bates, S. (2021). A gentle introduction to conformal prediction and distribution-free uncertainty quantification. arXiv preprint arXiv:2107.07511.

All other figures are taken from the same paper.

**Conformal inference recipe**

1. Train your model.
2. For each point $(x_i, y_i)$ in
the calibration set, compute the conformity score $s(x_i, y_i)$.
3. For a desired confidence
level $(1-\alpha)$ (e.g., $\alpha$ = 0.1 for 90% confidence), find the $\frac{(1 − \alpha)(n + 1)}{n}$-th quantile of
the calibration scores, where $n$ is the size of the calibration set.
4. For each test point $x$, construct the prediction set $C(x) = \{y : s(x, y) ≥ \text{quantile}\}$.

![illustration-conformal-inference.jpg](https://drive.google.com/uc?export=view&id=1tQJFM3JmzES1uYZrmlj-3m6btuXTX0_6)

## How will we implement conformal inference?

Conformal inference works for continuous and discrete data, and we will study **examples** of
- continuous outcome regression
- quantile regression
- discrete classification.

For each of these examples, we'll follow the following **steps**:
1. Load the data sets and split them into training, calibration, and test set. Train the predictors.
2. Compute _scores_ and _confidence sets_.
3. Summarise our results. We'll compute coverage rates, interval widths (regression) or prediction set sizes (classification), and visualise the prediction sets for selected examples.
4. Discuss strengths and limitations of the approach and compare results across different methods and data sets.

We'll work on steps 1 and 2 today and then finish the problem set up tomorrow.

# Setup

Load libraries and data sets - it's good practice to do this at the top of your script.

In [15]:
# Import all necessary libraries here.
import numpy as np
import warnings
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import StandardScaler # For normalizing regressors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Ridge, QuantileRegressor

# Import the wine and california_housing data sets from sklearn.datasets.
from sklearn.datasets import load_wine, fetch_california_housing

# Define functions to calculate scores and confidence sets

We'll define functions at the top because that's best practice. Fill in these functions as you work on the examples!

First, define the helper function `alphatilde(alpha, n)` for rounding the size cutoff appropriately. It computes
$$
\tilde\alpha = 1-\frac{\lceil(n+1)(1-\alpha)\rceil}{n}
$$

In [24]:
# Define alphatilde().
def alphatilde(alpha, n):
  return np.ceil((n+1)*(1-alpha))/n

Next define functions to compute the score thresholds and prediction sets for continuous regression, quantile regression, and discrete classification.

### Continuous outcomes, simple score

Define the conformity score as:
$$
s(x, y) = |y − \hat f(x)|
$$
where $\hat f(x)$ is the predicted value from the ridge regression model.

For evaluation, construct prediction intervals $[\hat f(x) − q, \hat f(x) + q]$ where $q$ is the appropriate quantile of calibration residuals.

### Quantile regression

Quantile regression estimates conditional quantiles of the outcome variable. If we construct an interval $[t_{\alpha/2}(x), t_{1-\alpha/2}(x)]$ with quantile regression, we would expect it to have approximately 90% coverage. So why do we add conformal inference to quantile regression?

Define the conformity score as:
$$
s(x, y) = \max(t_{\alpha/2}(x)− y, y − t_{1-\alpha/2}(x))
$$

where $t_{\alpha/2}(x)$ and $t_{1-\alpha/2}(x)$ are the predicted quantiles.

The conformity score measures how far the true value $y$ falls outside the predicted quantile interval, with negative scores indicating the point lies within the predicted interval.

Intuitively, $\hat q$ grows or shrinks the distance between the quantiles to achieve coverage:

![quantile-regression-illustration.jpg](https://drive.google.com/uc?export=view&id=1yl2-RCoPlPBfmaGYDc6WnRDlhtMZSmsv)

### Discrete classification

Define the conformity score as:

$$
s(x,y) = \sum_{y \prime} \mathbb{1} ( j(y|x) \ge  j(y^\prime|x ) ) \cdot \hat f(y|x)
$$

where $\mathbb{1}(\cdot)$ is the indicator function. $\hat f(y|x)$ represents the predicted probability of class $y$ given features $x$. $j(y, x)$ is the rank of $\hat f(y|x)$  across all possible labels $y$.

![prediction-sets-classification.jpg](https://drive.google.com/uc?export=view&id=1I0fMj-uZEkL5l7cT6U7WrgYn942o0_T0)

# Empirical examples

## Continuous regression

Load the `california_housing` data and split it into features X and labels Y.



For a documentation of the data, see: https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset

Split the data into three subsets:
- a **training set** (60%) for initial model fitting,
- a **calibaration set** (20%) used for conformal inference calibration,
- and a **validation set** (20%) for final evaluation.

In [16]:
# Load the data and separate the data set into features X and outcomes y.
data1 = fetch_california_housing()
X = data1["data"]
X = StandardScaler().fit_transform(X)
y = data1["target"]

In [17]:
# Split the data into training, calibration, and validation.
# Hint: Do this in two steps using train_test_split().
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_cal, X_val, y_cal, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

Implement a _ridge regression_ with penalty parameter selected via cross-validation. Use cross-validation on the training set to find the optimal penalty parameter. (See the "Supervised Learning" PS for guidance.)

In [18]:
# Define the parameter range.
alpha_values = np.logspace(-3, 3, 20)

# Run a grid search with cross-validation.
ridge_cv = GridSearchCV(Ridge(), {'alpha': alpha_values}, cv=10)
with warnings.catch_warnings(): # Suppress output of warnings
    warnings.simplefilter("ignore")
    ridge_cv.fit(X_train, y_train)

# Select the best model.
bestmodel = ridge_cv.best_estimator_

Compute the predictions on calibration and validation data sets.

In [19]:
# Compute the predictions on calibration and validation data sets.
# Make sure to store them for future reference.
y_calhat = bestmodel.predict(X_cal)
y_valhat = bestmodel.predict(X_val)

Compute the threshold as the quantile of the calibration scores. Then obtain the prediction sets for the validation data.

In [25]:
# Compute the scores.
# (where does the calibration scores come from: the absolute difference)
# calibration data set: y - yhat

# define a function computing the cutoff q hat for
def continuous_score_cutoff(y, yhat, alpha = .1):
    n = y.size
    cal_scores = np.abs(y - yhat)
    return  np.quantile(cal_scores, alphatilde(alpha, n))

# Compute the threshold q_hat based on scores and alphatilde().
cutoff = continuous_score_cutoff(y_cal, y_calhat)


In [28]:
# Then use the threshold to obtain the prediction sets for the validation data.

# Prediction sets for validation data: [y_valhat - cutoff, y_valhat + cutoff]
prediction_sets_val = np.array([y_valhat - cutoff, y_valhat + cutoff]).T




In [29]:
# Print the shape and the first few entries of the prediction sets to check your work.

print("Shape of prediction sets:", prediction_sets_val.shape)
print("First 5 entries of prediction sets:\n", prediction_sets_val[:5])

Shape of prediction sets: (4128, 2)
First 5 entries of prediction sets:
 [[ 0.83700263  3.07265437]
 [-0.40571951  1.82993222]
 [ 1.27247893  3.50813066]
 [ 0.7198914   2.95554314]
 [ 0.1460148   2.38166654]]


Calculate the average interval width and compute empirical coverage on the validation data.

In [34]:
# Average interval width
# The interval width is the difference between the upper and lower bounds of the prediction set.

interval_widths = prediction_sets_val[:, 1] - prediction_sets_val[:, 0]
average_width = np.mean(interval_widths)
print(f"Average interval width: {average_width:.4f}")
# f"": f-string to embed the value of variables directly within a string
# .4f four digits

n_val = confidencesets_simple.shape[0]
average_interval_width_simple = sum((confidencesets_simple[i][1] - confidencesets_simple[i][0])for i in range(n_val)) /n_val
print("Average interval width:", average_interval_width_simple)
print("\nContinuous Regression Confidence Sets Shape:", confidencesets_simple.shape)
print("Continuous Regression Confidence Sets (first 5):")
print(confidencesets_simple[:5])

# Coverage
coverage_simple = sum((y_val[i] >= confidencesets_simple[i][0] and y_val[i] <= confidencesets_simple[i][1]) for i in range(n_val)) /n_val
print("Coverage:", coverage_simple)


Average interval width: 2.2357


NameError: name 'confidencesets_simple' is not defined

**What do set sizes tell us?**

- **Set size:**
- **Spread of set size** (i.e., some sets are small, others are large):

1. we want smaller intervals (better prediction)
2. shorter interval length, better prediction

Plot prediction intervals for a subset of validation points and compare them to the true values.

In [7]:
# Create your plot here to visualise your work. Hint: Use matplotlib.pyplot.scatter() and matplotlib.pyplot.plot()

Plot the distribution of interval width over the validation set. What do you notice?

In [8]:
# Create your plot here. Hint: Use matplotlib.pyplot.hist()

## Quantile regression

Continue using the `california_housing` data.

Fit quantile regression models for quantiles $\tau_{\alpha/2}$ and $\tau_{1-\alpha/2}$ (e.g., 0.05 and 0.95 for $\alpha$ = 0.1). For simplicity, use fixed L1 regularisation and no tuning.
Note that you can use scikit-learn’s `QuantileRegressor`.

In [35]:
# Fit quantile regression models for lower and upper quantiles.
alpha = .1 # alpha refers to the significance level for the quantile regression here, not to conformal inference!
qr_lower = QuantileRegressor(quantile=alpha/2, alpha=0.01)
qr_lower.fit(X_train, y_train)

qr_upper = QuantileRegressor(quantile= 1-alpha/2, alpha=0.01)
qr_upper.fit(X_train, y_train)

QuantileRegressor(alpha=0.01, quantile=0.95)

Compute the predictions on calibration and validation data sets.

In [36]:
# Compute the predictions on calibration and validation data sets for both quantiles.
# Make sure to store them for future reference.
y_callower = qr_lower.predict(X_cal)
y_vallower = qr_lower.predict(X_val)

y_calupper = qr_upper.predict(X_cal)
y_valupper = qr_upper.predict(X_val)

Compute the threshold as the quantile of the calibration scores. Then obtain the prediction sets for the validation data.

In [38]:
# Compute the thresholds. (check the conformality equation for quantile regression)
# WHY use Quantile regression in conformal inference: to be precise on the confidence intervals
# to get guaranteed coverage
# (conditional coverage)
# Compute the threshold q_hat based on scores and alphatilde().
cutoff_q_lower = continuous_score_cutoff(y_cal, y_callower)
cutoff_q_upper = continuous_score_cutoff(y_cal, y_calupper)


In [ ]:
# Then use the thresholds to obtain the prediction sets for the validation data.
# Compute the threshold q_hat based on scores and alphatilde().
min-lower, max+upper

prediction_sets_val


In [ ]:
# Print the shape and the first few entries of the prediction sets to check your work.
print("Shape of prediction sets:", prediction_sets_val.shape)
print("First 5 entries of prediction sets:\n", prediction_sets_val[:5])


Compare coverage and interval widths with the ridge regression approach from the
previous problem. Analyze whether quantile regression provides more adaptive intervals and discuss the trade-offs between the two approaches.

In [ ]:
# Average interval width

# Coverage

Plot prediction intervals for a subset of validation points and compare them to the true values.

In [ ]:
# Create your plot here.

Plot the distribution of interval width over the validation set. What do you notice? Compare your results to the simple regression.

In [ ]:
# Create your plot here.

## Discrete classification

Load the `wine` data from the skikit-learn package and split it into features $X$ and labels $Y$.

For a documentation of the data, see: https://scikit-learn.org/stable/datasets/toy_dataset.html#wine-recognition-dataset.

In [10]:
# Load the data and separate the data set into features X and outcomes y.
data2 = load_wine()
X = data2["data"]
X = StandardScaler().fit_transform(X)
y = data2["target"]

NameError: name 'load_wine' is not defined

In [ ]:
# Split into train and test set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
# Split the test set into calibration and validation.
X_cal, X_val, y_cal, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

### Logistic regression

Fit a penalized logistic regression model using scikit-learn’s `LogisticRegression` with L2 regularization. Use cross-validation on the training set to find the optimal penalty parameter $\lambda$  that minimizes classification error.

This will give the scores for _conformal inference_.

In [ ]:
# Define the parameter range.
param_range = np.logspace(-4, 2, 20)

# Run a grid search with cross-validation.
logistic_cv = GridSearchCV(LogisticRegression(), {'C': param_range}, cv=10)
with warnings.catch_warnings(): # Suppress output of warnings
    warnings.simplefilter("ignore")
    logistic_cv.fit(X_train, y_train)

# Select the best model.
bestmodel = logistic_cv.best_estimator_

Compute the predictions on calibration and validation data sets.

In [ ]:
# Compute the predictions on calibration and validation data sets.
# Make sure to store them for future reference.
y_calproba = bestmodel.predict_proba(X_cal) # proba returns the probability estimates for each class rather than just the predicted class label
y_valproba = bestmodel.predict_proba(X_val)

Compute the threshold as the quantile of the calibration scores. Then obtain the prediction sets for the validation data.

In [11]:
# Compute the thresholds.
# Then use the thresholds to obtain the prediction sets for the validation data.

In [12]:
# Print the shape and the first few entries of the prediction sets to check your work.

Evaluate the method on the validation data set by computing:
- average set size (mean
number of labels in the prediction sets)
- coverage (fraction
of test points where the true label is in the prediction set)
- conditional coverage (coverage rates
for each true class): Is the coverage at least 90% for each type of wine?

In [13]:
# Average set size

# Coverage: is y_val[i] contained in confidencesets_classification[i]?

# Conditional coverage

### k-Nearest Neighbours

Repeat the conformal inference procedure from the previous problem, but replace the
penalized logistic regression with **k-nearest neighbors classification**. Use cross-validation on the training set to select the optimal number of neighbors $k$. Use the same conformity score definition as in the previous problem. Compare the coverage and efficiency (average prediction set size) with the logistic regression results, and discuss any differences in performance between the two methods.

In [14]:
# Define the parameter range.
k_range = np.arange(1, 50)

# Run a grid search with cross-validation.
kNN_cv = GridSearchCV(KNeighborsClassifier(), {'n_neighbors': k_range}, cv=10, scoring="accuracy")
with warnings.catch_warnings(): # Suppress output of warnings
    warnings.simplefilter("ignore")
    kNN_cv.fit(X_train, y_train)
print(kNN_cv.best_params_)

# Select the best model.
bestkNN = kNN_cv.best_estimator_

NameError: name 'np' is not defined

Compute the predictions on calibration and validation data sets.

In [ ]:
# Compute the predictions on calibration and validation data sets.
# Make sure to store them for future reference.

Compute the threshold as the quantile of the calibration scores. Then obtain the prediction sets for the validation data.

In [ ]:
# Compute the thresholds.
# Then use the thresholds to obtain the prediction sets for the validation data.

In [ ]:
# Print the shape and the first few entries of the prediction sets to check your work.

Evaluate the method on the validation data set by computing:
- average set size (mean
number of labels in the prediction sets)
- coverage (fraction
of test points where the true label is in the prediction set)
- conditional coverage (coverage rates
for each true class): Is the coverage at least 90% for each type of wine?

In [ ]:
# Average set size

# Coverage: is y_val[i] contained in confidencesets_classification[i]?

# Conditional coverage

Compare your results to the Logistic Regression model. What do you notice?